In [ ]:
#Display Data

In [10]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import hashlib
import datetime
import webbrowser
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import openpyxl
from PIL import Image, ImageTk

# Load dataset
df = pd.read_csv(r"D:\final\succ\succ\instagram.csv")

# Select features and target
X = df.drop('fake', axis=1)
y = df['fake']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Define Block class to represent individual blocks in the blockchain
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp  # Added timestamp attribute
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(str(self.index).encode() +
                              str(self.timestamp).encode() +
                              str(self.data).encode() +
                              str(self.previous_hash).encode()).hexdigest()

# Define Blockchain class to manage the blockchain
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, datetime.datetime.now(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

# Function to detect fake or real profile
def detect_fake_or_real_profile(profile_data):
    # Use the trained model to predict if the profile is fake or real
    prediction = model.predict([profile_data])
    return "Real" if prediction == 0 else "Fake"

# Function to register a new user
def register(username, password):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    sheet.append([username, password])
    wb.save("login.xlsx")
    messagebox.showinfo("Success", "Registration successful!")

# Function to check if username exists
def username_exists(username):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    for row in sheet.iter_rows(values_only=True):
        if row[0] == username:
            return True
    return False

# Function to validate login
def login(username, password):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    for row in sheet.iter_rows(values_only=True):
        if row[0] == username and row[1] == password:
            return True
    return False

class FakeProfileDetectorApp(tk.Frame):
    def __init__(self, root=None):
        super().__init__(root)
        self.root = root
        self.bg_photo = None
        self.blockchain = Blockchain()
        self.create_widgets()  # Corrected indentation here

    def create_widgets(self):
        # Add a Canvas widget to place background image410x630
        canvas = tk.Canvas(self.root, width=410, height=630)
        canvas.pack()

        # Load the background image
        bg_image = Image.open("bg.jpg")  # Corrected filename
        bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=bg_photo)
        canvas.image = bg_photo  # Keep a reference to avoid garbage collection

        # Create labels for the input fields with increased font size and changed color
        label_font = ('Times New Roman', 15)
        label_font1 = ('Times New Roman', 12)
        label_font2 = ('Times New Roman', 10)
        label_color = 'white'  # Change the color here

        canvas.create_text(210, 40, text="Fake Profile Detection With Blockchain", font=label_font, fill=label_color)
        canvas.create_text(200, 80, text="If There's Data Give 1 or Give 0", font=label_font1, fill=label_color)

        canvas.create_text(110, 140, text="Profile", font=label_font2, fill=label_color)
        canvas.create_text(110, 200, text="Username", font=label_font2, fill=label_color)
        canvas.create_text(110, 260, text="Link", font=label_font2, fill=label_color)
        canvas.create_text(110, 320, text="Posts", font=label_font2, fill=label_color)
        canvas.create_text(110, 380, text="followers", font=label_font2, fill=label_color)
        canvas.create_text(110, 440, text="follows", font=label_font2, fill=label_color)

        self.profile = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 140, window=self.profile)

        self.username = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 200, window=self.username)

        self.URL = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 260, window=self.URL)

        self.posts = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 320, window=self.posts)

        self.followers = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 380, window=self.followers)

        self.follows = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 440, window=self.follows)

        self.detect_button = tk.Button(canvas, text="Detect Fake Profile", command=self.detect_fake_profile, width=35, height=1)
        canvas.create_window(200, 520, window=self.detect_button)

        display_button = tk.Button(canvas, text="Display Blockchain Data", command=self.display_blockchain, width=35, height=1)
        canvas.create_window(200, 560, window=display_button)

        share_button = tk.Button(canvas, text="Share Blockchain Data", command=self.share_blockchain_data, width=35, height=1)
        canvas.create_window(200, 600, window=share_button)

    def detect_fake_profile(self):
        profile = int(self.profile.get())
        username = int(self.username.get())
        URL = int(self.URL.get())
        posts = int(self.posts.get())
        followers = int(self.followers.get())
        follows = int(self.follows.get())

        profile_data = [profile, username, URL, posts, followers, follows]

        result = detect_fake_or_real_profile(profile_data)

        messagebox.showinfo("Detection Result", f"Profile is {result}")

        self.blockchain.add_block(Block(len(self.blockchain.chain), datetime.datetime.now(), profile_data, self.blockchain.get_latest_block().hash))

    def share_blockchain_data(self):
        blockchain_data = ""
        for block in self.blockchain.chain:
            blockchain_data += f"Block {block.index}:\nTimestamp: {block.timestamp}\nData: {block.data}\nHash: {block.hash}\nPrevious Hash: {block.previous_hash}\n\n"

        whatsapp_url = f"https://web.whatsapp.com/send?text={blockchain_data}"

        webbrowser.open(whatsapp_url)

    def display_blockchain(self):
        self.display_window = tk.Toplevel(self.root)
        self.display_window.title("Blockchain Data")
        self.display_window.geometry("600x400")  # Set the size of the display window

        data_text = tk.Text(self.display_window, height=20, width=60)  # Increase the height and width of the Text widget
        data_text.pack(padx=10, pady=10)

        for block in self.blockchain.chain:
            data_str = f"Block: {block.index},\nTimestamp: {block.timestamp}\nData: {block.data},\nPrevious Hash: {block.previous_hash}\n\n"
            data_text.insert("end", data_str)


class LoginRegisterApp(tk.Frame):
    def __init__(self, root=None):
        super().__init__(root)
        self.root = root
        self.bg_photo = None
        self.login_widgets = None
        self.register_widgets = None
        self.create_login_widgets()

    def create_login_widgets(self):
        if self.login_widgets:
            self.login_widgets.destroy()
        if self.register_widgets:
            self.register_widgets.destroy()

        self.login_widgets = tk.Frame(self.root)
        self.login_widgets.pack()

        label_font3 = ('Times New Roman', 18)
        label_font4 = ('Times New Roman', 12)
        label_color1 = 'white'

        canvas = tk.Canvas(self.login_widgets, width=500, height=800)
        canvas.pack()

        bg_image = Image.open("bg.jpg")
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=self.bg_photo)

        canvas.create_text(200, 40, text="Login Page", font=label_font3, fill=label_color1)

        self.username_label_login = canvas.create_text(110, 120, text="Username", font=label_font4, fill=label_color1)
        self.password_label_login = canvas.create_text(110, 180, text="Password", font=label_font4, fill=label_color1)

        self.username_entry_login = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(290, 120, window=self.username_entry_login)

        self.password_entry_login = tk.Entry(canvas, highlightthickness=0, relief='flat', show="*")
        canvas.create_window(290, 180, window=self.password_entry_login)

        login_button = tk.Button(canvas, text="Login", command=self.login, width=20, height=2)
        canvas.create_window(200, 500, window=login_button)  # Adjusted y-coordinate here

        switch_to_register_button = tk.Button(canvas, text="Register", command=self.switch_to_register, width=20, height=2)
        canvas.create_window(200, 560, window=switch_to_register_button)

    def create_register_widgets(self):
        if self.login_widgets:
            self.login_widgets.destroy()
        if self.register_widgets:
            self.register_widgets.destroy()

        self.register_widgets = tk.Frame(self.root)
        self.register_widgets.pack()

        label_font3 = ('Times New Roman', 18)
        label_font4 = ('Times New Roman', 12)
        label_color1 = 'white'

        canvas = tk.Canvas(self.register_widgets, width=500, height=800)
        canvas.pack()

        bg_image = Image.open("bg.jpg")
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=self.bg_photo)

        canvas.create_text(200, 40, text="Register Page", font=label_font3, fill=label_color1)

        self.username_label_register = canvas.create_text(110, 120, text="Username", font=label_font4, fill=label_color1)
        self.password_label_register = canvas.create_text(110, 180, text="Password", font=label_font4, fill=label_color1)

        self.username_entry_register = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(290, 120, window=self.username_entry_register)

        self.password_entry_register = tk.Entry(canvas, highlightthickness=0, relief='flat', show="*")
        canvas.create_window(290, 180, window=self.password_entry_register)

        register_button = tk.Button(canvas, text="Register", command=self.register, width=20, height=2)
        canvas.create_window(200, 500, window=register_button)  # Adjusted y-coordinate here

        switch_to_login_button = tk.Button(canvas, text="Login", command=self.switch_to_login, width=20, height=2)
        canvas.create_window(200, 560, window=switch_to_login_button)

    def switch_to_register(self):
        self.create_register_widgets()

    def switch_to_login(self):
        self.create_login_widgets()

    def login(self):
        username = self.username_entry_login.get()
        password = self.password_entry_login.get()
        if not username_exists(username):
            messagebox.showwarning("Login", "You need to register first")
        elif login(username, password):
            messagebox.showinfo("Login", "Login Successful")
            self.root.destroy()
            root = tk.Tk()
            app = FakeProfileDetectorApp(root)
            app.mainloop()
        else:
            messagebox.showerror("Login", "Please enter valid data")

    def register(self):
        username = self.username_entry_register.get()
        password = self.password_entry_register.get()
        if not username or not password:
            messagebox.showerror("Registration", "Please enter valid data")
        elif username_exists(username):
            messagebox.showerror("Registration", "Username already exists")
        else:
            register(username, password)
            messagebox.showinfo("Registration", "Registration Successful")


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("410x630")
    app = LoginRegisterApp(root)
    app.create_login_widgets()
    app.mainloop()


C:\Users\sanam\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\sanam\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
#Hide data by encrypt

In [11]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import hashlib
import datetime
import webbrowser
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import openpyxl
from PIL import Image, ImageTk
from cryptography.fernet import Fernet  # Import Fernet for encryption

# Load dataset
df = pd.read_csv(r"D:\final\succ\succ\instagram.csv")

# Select features and target
X = df.drop('fake', axis=1)
y = df['fake']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Define Block class to represent individual blocks in the blockchain
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(str(self.index).encode() +
                              str(self.timestamp).encode() +
                              str(self.data).encode() +
                              str(self.previous_hash).encode()).hexdigest()

# Define Blockchain class to manage the blockchain
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, datetime.datetime.now(), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

# Function to detect fake or real profile
def detect_fake_or_real_profile(profile_data):
    # Use the trained model to predict if the profile is fake or real
    prediction = model.predict([profile_data])
    return "Real" if prediction == 0 else "Fake"

# Function to register a new user
def register(username, password):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    sheet.append([username, password])
    wb.save("login.xlsx")
    messagebox.showinfo("Success", "Registration successful!")

# Function to check if username exists
def username_exists(username):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    for row in sheet.iter_rows(values_only=True):
        if row[0] == username:
            return True
    return False

# Function to validate login
def login(username, password):
    wb = openpyxl.load_workbook("login.xlsx")
    sheet = wb.active
    for row in sheet.iter_rows(values_only=True):
        if row[0] == username and row[1] == password:
            return True
    return False

class FakeProfileDetectorApp(tk.Frame):
    def __init__(self, root=None):
        super().__init__(root)
        self.root = root
        self.bg_photo = None
        self.blockchain = Blockchain()
        self.key = Fernet.generate_key()  # Generate a symmetric encryption key
        self.fernet = Fernet(self.key)  # Create a Fernet instance with the key
        self.create_widgets()  # Corrected indentation here

    def create_widgets(self):
        # Add a Canvas widget to place background image410x630
        canvas = tk.Canvas(self.root, width=410, height=630)
        canvas.pack()

        # Load the background image
        bg_image = Image.open("bg.jpg")  # Corrected filename
        bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=bg_photo)
        canvas.image = bg_photo  # Keep a reference to avoid garbage collection

        # Create labels for the input fields with increased font size and changed color
        label_font = ('Times New Roman', 15)
        label_font1 = ('Times New Roman', 12)
        label_font2 = ('Times New Roman', 10)
        label_color = 'white'  # Change the color here

        canvas.create_text(210, 40, text="Fake Profile Detection With Blockchain", font=label_font, fill=label_color)
        canvas.create_text(200, 80, text="If There's Data Give 1 or Give 0", font=label_font1, fill=label_color)

        canvas.create_text(110, 140, text="Profile", font=label_font2, fill=label_color)
        canvas.create_text(110, 200, text="Username", font=label_font2, fill=label_color)
        canvas.create_text(110, 260, text="Link", font=label_font2, fill=label_color)
        canvas.create_text(110, 320, text="Posts", font=label_font2, fill=label_color)
        canvas.create_text(110, 380, text="followers", font=label_font2, fill=label_color)
        canvas.create_text(110, 440, text="follows", font=label_font2, fill=label_color)

        self.profile = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 140, window=self.profile)

        self.username = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 200, window=self.username)

        self.URL = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 260, window=self.URL)

        self.posts = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 320, window=self.posts)

        self.followers = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 380, window=self.followers)

        self.follows = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(250, 440, window=self.follows)

        self.detect_button = tk.Button(canvas, text="Detect Fake Profile", command=self.detect_fake_profile, width=35, height=1)
        canvas.create_window(200, 520, window=self.detect_button)

        display_button = tk.Button(canvas, text="Display Blockchain Data", command=self.display_blockchain, width=35, height=1)
        canvas.create_window(200, 560, window=display_button)

        share_button = tk.Button(canvas, text="Share Blockchain Data", command=self.share_blockchain_data, width=35, height=1)
        canvas.create_window(200, 600, window=share_button)

    def detect_fake_profile(self):
        profile = int(self.profile.get())
        username = int(self.username.get())
        URL = int(self.URL.get())
        posts = int(self.posts.get())
        followers = int(self.followers.get())
        follows = int(self.follows.get())

        profile_data = [profile, username, URL, posts, followers, follows]

        result = detect_fake_or_real_profile(profile_data)

        messagebox.showinfo("Detection Result", f"Profile is {result}")

        self.blockchain.add_block(Block(len(self.blockchain.chain), datetime.datetime.now(), profile_data, self.blockchain.get_latest_block().hash))

    def share_blockchain_data(self):
        blockchain_data = ""
        for block in self.blockchain.chain:
            encrypted_data = self.fernet.encrypt(str(block.data).encode())  # Encrypt the data
            blockchain_data += f"Block {block.index}:\nTimestamp: {block.timestamp}\nData: {encrypted_data}\nHash: {block.hash}\nPrevious Hash: {block.previous_hash}\n\n"

        whatsapp_url = f"https://web.whatsapp.com/send?text={blockchain_data}"

        webbrowser.open(whatsapp_url)
        
    def display_blockchain(self):
        self.display_window = tk.Toplevel(self.root)
        self.display_window.title("Blockchain Data")
        self.display_window.geometry("600x600")  # Set the size of the display window

        data_text = tk.Text(self.display_window, height=30, width=60)  # Increase the height and width of the Text widget
        data_text.pack(padx=10, pady=10)

        for block in self.blockchain.chain:
            encrypted_data = self.fernet.encrypt(str(block.data).encode())  # Encrypt the data
            data_str = f"Block: {block.index},\nData: {encrypted_data},\nTimestamp: {block.timestamp},\nPrevious Hash: {block.previous_hash}\n\n"
            data_text.insert("end", data_str)


class LoginRegisterApp(tk.Frame):
    def __init__(self, root=None):
        super().__init__(root)
        self.root = root
        self.bg_photo = None
        self.login_widgets = None
        self.register_widgets = None
        self.create_login_widgets()

    def create_login_widgets(self):
        if self.login_widgets:
            self.login_widgets.destroy()
        if self.register_widgets:
            self.register_widgets.destroy()

        self.login_widgets = tk.Frame(self.root)
        self.login_widgets.pack()

        label_font3 = ('Times New Roman', 18)
        label_font4 = ('Times New Roman', 12)
        label_color1 = 'white'

        canvas = tk.Canvas(self.login_widgets, width=500, height=800)
        canvas.pack()

        bg_image = Image.open("bg.jpg")
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=self.bg_photo)

        canvas.create_text(200, 40, text="Login Page", font=label_font3, fill=label_color1)

        self.username_label_login = canvas.create_text(110, 120, text="Username", font=label_font4, fill=label_color1)
        self.password_label_login = canvas.create_text(110, 180, text="Password", font=label_font4, fill=label_color1)

        self.username_entry_login = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(290, 120, window=self.username_entry_login)

        self.password_entry_login = tk.Entry(canvas, highlightthickness=0, relief='flat', show="*")
        canvas.create_window(290, 180, window=self.password_entry_login)

        login_button = tk.Button(canvas, text="Login", command=self.login, width=20, height=2)
        canvas.create_window(200, 500, window=login_button)  # Adjusted y-coordinate here

        switch_to_register_button = tk.Button(canvas, text="Register", command=self.switch_to_register, width=20, height=2)
        canvas.create_window(200, 560, window=switch_to_register_button)

    def create_register_widgets(self):
        if self.login_widgets:
            self.login_widgets.destroy()
        if self.register_widgets:
            self.register_widgets.destroy()

        self.register_widgets = tk.Frame(self.root)
        self.register_widgets.pack()

        label_font3 = ('Times New Roman', 18)
        label_font4 = ('Times New Roman', 12)
        label_color1 = 'white'

        canvas = tk.Canvas(self.register_widgets, width=500, height=800)
        canvas.pack()

        bg_image = Image.open("bg.jpg")
        self.bg_photo = ImageTk.PhotoImage(bg_image)
        canvas.create_image(0, 0, anchor=tk.NW, image=self.bg_photo)

        canvas.create_text(200, 40, text="Register Page", font=label_font3, fill=label_color1)

        self.username_label_register = canvas.create_text(110, 120, text="Username", font=label_font4, fill=label_color1)
        self.password_label_register = canvas.create_text(110, 180, text="Password", font=label_font4, fill=label_color1)

        self.username_entry_register = tk.Entry(canvas, highlightthickness=0, relief='flat')
        canvas.create_window(290, 120, window=self.username_entry_register)

        self.password_entry_register = tk.Entry(canvas, highlightthickness=0, relief='flat', show="*")
        canvas.create_window(290, 180, window=self.password_entry_register)

        register_button = tk.Button(canvas, text="Register", command=self.register, width=20, height=2)
        canvas.create_window(200, 500, window=register_button)  # Adjusted y-coordinate here

        switch_to_login_button = tk.Button(canvas, text="Login", command=self.switch_to_login, width=20, height=2)
        canvas.create_window(200, 560, window=switch_to_login_button)

    def switch_to_register(self):
        self.create_register_widgets()

    def switch_to_login(self):
        self.create_login_widgets()

    def login(self):
        username = self.username_entry_login.get()
        password = self.password_entry_login.get()
        if not username_exists(username):
            messagebox.showwarning("Login", "You need to register first")
        elif login(username, password):
            #messagebox.showinfo("Login", "Login Successful")
            self.root.destroy()
            root = tk.Tk()
            app = FakeProfileDetectorApp(root)
            app.mainloop()
        else:
            messagebox.showerror("Login", "Please enter valid data")

    def register(self):
        username = self.username_entry_register.get()
        password = self.password_entry_register.get()
        if not username or not password:
            messagebox.showerror("Registration", "Please enter valid data")
        elif username_exists(username):
            messagebox.showerror("Registration", "Username already exists")
        else:
            register(username, password)
            messagebox.showinfo("Registration", "Registration Successful")


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("410x630")
    app = LoginRegisterApp(root)
    app.create_login_widgets()
    app.mainloop()


C:\Users\sanam\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\sanam\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
